This notebook calculates the distribution of completly correct, wrong and mixed examples and the sentence length in the datasets. 

In [1]:
import pickle
import torch
import pandas as pd

In [2]:
def load_data(model,mode, dataset, split):
#load predicted and gold bounding boxes
        
    try:

        #the predicted bounding box
        with open(r"/home/users/fschreiber/project/bboxes_"+model+"/"+dataset+"/"+split+"_pred_bbox_list.p","rb") as f:
            pred_bbox_list=list(pickle.load(f))

        if mode=="non_inc":
            #the target bounding box
            with open(r"/home/users/fschreiber/project/bboxes_noninc_"+model+"/"+dataset+"/"+split+"_pred_bbox_list.p","rb") as f:
                target_bbox_list=list(pickle.load(f))

        elif mode == "inc":
            #the target bounding box
            with open(r"/home/users/fschreiber/project/bboxes_"+model+"/"+dataset+"/"+split+"_target_bbox_list.p","rb") as f:
                target_bbox_list=list(pickle.load(f))
        else:
            print("The mode can only be non_inc or inc")
            return -1,-1,-1,-1,-1

        #the number of one sentence split up incrementally ("the left zebra" would have length 3)
        with open(r"/home/users/fschreiber/project/incremental_pickles/length_incremental_units/"+dataset+"_"+split+"_length_unit.p","rb") as f:
            inc_len=pickle.load(f)

        #the original model data split up incrementally
        data_model=torch.load("/home/users/fschreiber/project/ready_inc_data/"+dataset+"/"+dataset+"_"+split+".pth")

        #information if the object was found or not grouped by sentence
        with open(r"/home/users/fschreiber/project/binary_grouped/"+model+"/"+mode+"/"+dataset+split+".p","rb") as f:
            binary_grouped=pickle.load(f)

        #if non_inc the targetboxes need to be filed up to the complete length
        if mode=="non_inc":
            target_bbox_list=[x for x,y in zip(target_bbox_list,inc_len) for _ in range(y)]
            
        if model=="TVG":
            pred_bbox_list,target_bbox_list=TVG_prep(pred_bbox_list,target_bbox_list)
        return pred_bbox_list,target_bbox_list,inc_len,data_model,binary_grouped
    
    except FileNotFoundError as e:
        #print(e)
        
        return  -1,-1,-1,-1,-1
    

#TVG needs some extra adjustments to fit the same data format as Resc
def TVG_prep(pred_bbox_list,target_bbox_list):
    #print("TVG")
    for ind,(pred,targ) in enumerate (zip (pred_bbox_list,target_bbox_list)):

        pred=pred.view(1,-1)

        pred=xywh2xyxy(pred)
        pred=torch.clamp(pred,0,1)

        pred_bbox_list[ind]=pred

        targ=targ.view(1,-1)
        targ=xywh2xyxy(targ)

        target_bbox_list[ind]=targ
    return pred_bbox_list,target_bbox_list

#copied from TransVG needed to transform the bounding box vectors
def xywh2xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=-1)

In [3]:
pred_bbox_list,target_bbox_list,inc_len,model,binary_grouped=load_data("TVG","inc","unc","testB")

In [4]:
#group sentences that belong to one incremental unit
def group_by_increment(bbox_list,inc_len):
    counter=0
    group_list=[]
    for i in inc_len:
        
        group_list.append(bbox_list[counter:counter+i])
        counter=counter+i
    return group_list


pred_group=group_by_increment(pred_bbox_list,inc_len)
targ_group=group_by_increment(target_bbox_list,inc_len)
model_group=group_by_increment(model,inc_len)

In [5]:
#split up the list into entries where each guess is correct,wrong or mixed save the indices

def split_by_correct(binary_grouped):
    only_one=[]
    only_zero=[]
    mixed=[]
    #mixed_values=[]

    for index,entry in enumerate(binary_grouped):
        if all(p == 1 for p in entry):
            only_one.append(index)
        elif all (p ==0 for p in entry):
            only_zero.append(index)
        else:
            mixed.append(index)
            #mixed_values.append(entry)
    
    return only_one,only_zero,mixed


only_one,only_zero,mixed=split_by_correct(binary_grouped)


In [10]:
# Initialize three empty lists: one_all, zero_all, and mixed_all.
one_all, zero_all, mixed_all = ([] for i in range(3))

# Define variables for mode, model_input, split_list, and dataset_list.
mode = "inc"
model_input = "ReSc"
split_list = ["testB", "testA", "val", "test"]
dataset_list = ["unc", "unc+", "gref_umd", "gref"]

# Create an empty list to store found dataset splits and sentence lengths.
found_sets_list = []
sentence_length_list = []

# Iterate through each dataset in dataset_list and each split in split_list.
for file in dataset_list:
    for split in split_list:

        # Call the load_data function to load data for the given parameters.
        pred_bbox_list, target_bbox_list, inc_len, model, binary_grouped = load_data(model_input, mode, file, split)

        # Check if the file is not found and skip to the next iteration.
        if pred_bbox_list == -1 or target_bbox_list == -1 or inc_len == -1:
            pass
        else:
            # Print the combination of file and split.
            print(file + split)
            found_sets_list.append(file + split)

            # Split the binary_grouped data into three categories: only_one, only_zero, and mixed.
            only_one, only_zero, mixed = split_by_correct(binary_grouped)

            # Calculate and append the percentage of each category to one_all, zero_all, and mixed_all.
            one_all.append(len(only_one) / len(binary_grouped))
            zero_all.append(len(only_zero) / len(binary_grouped))
            mixed_all.append(len(mixed) / len(binary_grouped))

            # Calculate and append the average sentence length to sentence_length_list.
            sentence_length = []
            for sent in binary_grouped:
                sentence_length.append(len(sent))
            sentence_length_list.append(sum(sentence_length) / len(sentence_length))

# Create a dictionary to store the collected data.
data = {}
data[model_input] = found_sets_list

# Store the percentages of correct, wrong, and mixed predictions, and average sentence length 
data["Right"] = one_all
data["Wrong"] = zero_all
data["Mixed"] = mixed_all
data["Average Sentence Length"] = sentence_length_list


df = pd.DataFrame(data)
df.set_index(model_input, inplace=True)
df = df.round(2)

df


unctestB
unctestA
uncval
unc+testB
unc+testA
unc+val
gref_umdval
gref_umdtest
grefval


,Right,Wrong,Mixed,Average Sentence Length
ReSc,,,,
unctestB,0.44,0.24,0.32,3.63
unctestA,0.50,0.17,0.32,3.43
uncval,0.48,0.19,0.33,3.57
unc+testB,0.36,0.34,0.30,3.76
unc+testA,0.46,0.25,0.29,3.34
unc+val,0.42,0.28,0.31,3.59
gref_umdval,0.25,0.22,0.53,8.37
gref_umdtest,0.24,0.22,0.54,8.28
grefval,0.19,0.25,0.56,8.37
